In [1]:
# podjela po mjestu tvorbe
vokali = ['a', 'e', 'i', 'o', 'u']
sonanti = ['v', 'r', 'l', 'n', 'j', 'lj', 'nj', 'm']

# sonanti
alveolarni = ['r', 'l', 'n']
palatalni = ['j', 'lj', 'nj']
bilabijalni = ['m']
labio_dentalni = ['v']

# konsonanti
labialni = ['b', 'p', 'f'] # usneni
zubni = ['d', 't', 'z', 's', 'c']
palatalni = ['dž', 'č', 'đ', 'ć', 'ž', 'š'] # prednjenepcani
zadnjonepcani = ['k', 'g', 'h'] # velarni

In [2]:
# podjela po zvucnosti
zvucni = ['b', 'g', 'd', 'đ', 'ž', 'z', 'dž']
bezvucni = ['p', 'k', 't', 'ć', 'š', 's', 'č', 'h', 'f', 'c']

In [4]:
fonem_types = [vokali, sonanti, alveolarni, palatalni, bilabijalni, labio_dentalni, labialni, zubni, palatalni, zadnjonepcani, zvucni, bezvucni]

In [5]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
import re

In [7]:
target_sentences_df = pd.read_csv('/content/drive/MyDrive/PhD/target_sentences.csv')
target_sentences_df

,Text
0,još malo pa će izbori
1,naoblačilo se kao da će padati kiša
2,naša kola su stara 10 godina
3,otišao je u inostranstvo prošle godine i još s...
4,telefoni zvone po ceo dan i tebe traže
...,...
62,pojavio se u poslednjem trenutku
63,sanjao sam te pre neku noć
64,premijer je obećao viši standard
65,sutra je doček Nove Godine


In [8]:
sentence_list = []
words_list = []

for i in range(0,len(target_sentences_df)):

    sentence = target_sentences_df['Text'][i].lower()
    words = sentence.split(' ')

    for word in words:
      sentence_list.append(i)
      words_list.append(word)

data = pd.DataFrame({'sentence':sentence_list, 'word':words_list})
data

,sentence,word
0,0,još
1,0,malo
2,0,pa
3,0,će
4,0,izbori
...,...,...
453,65,doček
454,65,nove
455,65,godine
456,66,nadam


In [9]:
def extraxt_foneme(word):

    pattern = r'^[+-]?\d+(\.\d+)?$'
    if bool(re.match(pattern, word)):
      if word == '10':
        word = 'deset'
      if word == '27':
        word = 'dvadesetsedam'
      if word == '50':
        word = 'pedeset'
      if word == '100':
        word = 'sto'

    # Define the pattern to match 'dz' as one unit or any single letter
    pattern = r'dz|lj|nj|.'
    # Use findall to get all matches
    parts = re.findall(pattern, word)
    return parts

In [10]:
# Save the DataFrame to a CSV file
csv_file_path = "/content/drive/MyDrive/PhD/fonetic_features.csv"


for foneme_type in fonem_types:
  fonem_list = []

  for index, row in data.iterrows():
    word = row['word']
    counter = 0
    foneme = extraxt_foneme(word)

    for fonem in foneme:
      if fonem in foneme_type:
        counter += 1

    fonem_list.append(counter)

  list_name = [name for name in globals() if globals()[name] is foneme_type][0]
  data[list_name] = fonem_list

# Sum specific columns and create a new column 'sum_col'
data['konsonanti'] = data[['labialni', 'zubni', 'palatalni', 'zadnjonepcani']].sum(axis=1)
data['zvucni'] = data[['zvucni', 'vokali', 'sonanti']].sum(axis=1)

In [11]:
data

,sentence,word,vokali,sonanti,alveolarni,palatalni,bilabijalni,labio_dentalni,labialni,zubni,zadnjonepcani,zvucni,bezvucni,konsonanti
0,0,još,1,1,0,1,0,0,0,0,0,2,1,1
1,0,malo,2,2,1,0,1,0,0,0,0,4,0,0
2,0,pa,1,0,0,0,0,0,1,0,0,1,1,1
3,0,će,1,0,0,1,0,0,0,0,0,1,1,1
4,0,izbori,3,1,1,0,0,0,1,1,0,6,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,65,doček,2,0,0,1,0,0,0,1,1,3,2,3
454,65,nove,2,2,1,0,0,1,0,0,0,4,0,0
455,65,godine,3,1,1,0,0,0,0,1,1,6,0,2
456,66,nadam,2,2,1,0,1,0,0,1,0,5,0,1


In [12]:
data.to_csv('/content/drive/MyDrive/PhD/fonetic_features1.csv', index=False)